In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_excel('./data/data.xlsx')

In [19]:
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,199801010001,1998,1,1,NaN,0,NaT,34,Burundi,11,Sub-Saharan Africa,Bujumbura Mairie,Bujumbura,-3.375828,29.364107,1.0,0,The incident occurred at Bujumbura Airport.,01/01/1998: Hutu Rebels attacked a Burundi mil...,1,1,1,1.0,1.0,Insurgency/Guerilla Action,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,4,Military,34.0,"Military Personnel (soldiers, troops, officers...",Burundi Military,Burundi Military at Bujumbura Airport,34.0,Burundi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hutu extremists,NaN,NaN,NaN,NaN,NaN,Unknown,0.0,NaN,NaN,0,100.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,5.0,Unknown Gun Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,0.0,100.0,6.0,0.0,NaN,-9,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"“Burundi Rebels, Ex-Rwandan Army Soldiers Blam...",“Burundi--Attack Reported on Bujumbura Airport...,NaN,CETIS,0,1,0,1,NaN
1,199801010002,1998,1,1,NaN,0,NaT,167,Russia,9,Eastern Europe,Moscow (Federal City),Moscow,55.751377,37.579914,1.0,0,NaN,"01/01/1998: In Russia, a small bomb hidden in ...",1,1,1,0.0,NaN,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,19,Transportation,102.0,Subway,Moscow Metro System,An underground passage in the Moscow Metro,167.0,Russia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,Unknown,0.0,NaN,NaN,0,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,14.0,Time Fuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"“Bomb injures 3 in Moscow subway system,” The ...","“Bomb injures 3 in Moscow subway,” Charleston ...","“Bomb Injures 3 Workers in Moscow Metro,” Los ...",CETIS,-9,-9,0,-9,NaN
2,199801010003,1998,1,1,NaN,0,NaT,603,United Kingdom,8,Western Europe,Northern Ireland,Belfast,54.607712,-5.956210,1.0,0,Belfast (Capital City),01/01/1998: The breakaway Loyalist Volunteer F...,1,1,1,0.0,NaN,NaN,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,Civilian,Eddie Trainor,233.0,Northern Ireland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loyalist Volunteer Forces (LVF),NaN,NaN,NaN,NaN,NaN,Unknown,0.0,NaN,NaN,0,NaN,0.0,1,8.0,Personal claim,NaN,NaN,NaN,NaN,NaN,NaN,-9.0,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,“Protestant gunmen kill Catholic in New Ye

24.0

In [4]:
xlsx = [201701090031,201702210037,201703120023,201705050009,201705050010,201707010028,201707020006,201708110018,201711010006,201712010003]

In [24]:
df.gname[1729]

'The Northern Alliance (or United Islamic Front for Salvation of Afghanistan - UIFSA)'

In [23]:
df[(df.country_txt == 'Afghanistan') & (df.gname != 'Unknown')]['gname'].head(20)

1729    The Northern Alliance (or United Islamic Front...
1749    The Northern Alliance (or United Islamic Front...
1754    The Northern Alliance (or United Islamic Front...
1755    The Northern Alliance (or United Islamic Front...
3686    The Northern Alliance (or United Islamic Front...
4175                                              Taliban
4258                                     Opposition Group
4308                                     Opposition Group
4311                                     Opposition Group
4312                                     Opposition Group
4357                                     Opposition Group
5269                                              Taliban
5609                                              Taliban
5677                                              Taliban
6443                                             Al-Qaida
6464                                              Taliban
6536                                              Taliban
6806          

In [5]:
df[df.eventid.isin(xlsx)]

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
103533,201701090031,2017,1,9,NaN,0,NaT,95,Iraq,10,Middle East & North Africa,Saladin,Tikrit,35.032500,43.646389,1.0,1,The incident occurred in the Hamreen mountains.,01/09/2017: An explosive device detonated near...,1,1,1,0.0,NaN,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,14,Private Citizens & Property,67.0,Unnamed Civilian/Unspecified,Not Applicable,Civilians,95.0,Iraq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,"The specific motive is unknown; however, sourc...",0.0,NaN,NaN,0,-99.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,16.0,Unknown Explosive Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Bomb blast kills 15 persons while fleeing eas...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
104775,201702210037,2017,2,21,NaN,0,NaT,4,Afghanistan,6,South Asia,Uruzgan,Shah Mansour,32.656200,65.903198,1.0,0,The incident occurred in Tarin Kot district.,02/21/2017: Assailants attacked security posts...,1,1,1,0.0,NaN,NaN,0.0,1,0,9,Unknown,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Afghan Police,Posts,4.0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,0.0,10.0,15.0,0.0,10.0,-9,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""4 policemen killed, 5 wounded in Uruzgan clas...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
105390,201703120023,2017,3,12,NaN,0,NaT,1004,South Sudan,11,Sub-Saharan Africa,Western Bieh,Unknown,8.650116,30.987207,4.0,0,NaN,03/12/2017: Assailants attacked military posit...,1,1,1,0.0,NaN,NaN,0.0,1,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,4,Military,34.0,"Military Personnel (soldiers, troops, officers...",Sudan People's Liberation Army (SPLA),Soldiers,1004.0,South Sudan,14.0,Private Citizens & Property,76.0,House/Apartment/Residence,Not Applicable,Houses,1004.0,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,"The specific motive is unknown; however, sourc...",0.0,NaN,NaN,0,-99.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Incendiary,18.0,Arson/Fire,13.0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,0.0,0.0,56.0,0.0,0.0,1,3.0,Minor (likely < $1 million),-99.0,Houses were looted and damaged during this att...,0.0,NaN,NaN,NaN,NaN,NaN,NaN

In [3]:
# 删除缺失值太多的列
drop_col = list(df.isnull().sum()[df.isnull().sum() >= 80000].index)
df = df.drop(drop_col, axis=1)

In [4]:
# 删除描述列
drop_col = [i for i in df.columns if 'txt' in i]
df = df.drop(drop_col, axis=1)

In [5]:
# 填充缺失值

# one-hot类型
df.provstate = df.provstate.fillna('No Provstate')
df.city = df.city.fillna('No City')
df.specificity = df.specificity.fillna(-1)
df.doubtterr = df.doubtterr.fillna(-1)
df.multiple = df.multiple.fillna(-1)
df.targsubtype1 = df.targsubtype1.fillna(-1)
df.corp1 = df.corp1.fillna('No Corp1')
df.target1 = df.target1.fillna('No Target1')
df.natlty1 = df.natlty1.fillna(-1)
df.guncertain1 = df.guncertain1.fillna(-1)
df.weapsubtype1 = df.weapsubtype1.fillna(-1)
df.propextent = df.propextent.fillna(4)
df.ishostkid = df.ishostkid.fillna(-9)

# txt类型
df.location = df.location.fillna('Unknown')
df.summary = df.summary.fillna('Unknown')
df.motive = df.motive.fillna('Unknown')
df.propcomment = df.propcomment.fillna('Unknown')
df.weapdetail = df.weapdetail.fillna('Unknown')
df.scite1 = df.scite1.fillna('Unknown')
df.scite2 = df.scite2.fillna('Unknown')
df.scite3 = df.scite3.fillna('Unknown')

# 连续型
df.latitude = df.latitude.fillna(df.latitude.mean())
df.longitude = df.longitude.fillna(df.longitude.mean())
df.nperps = df.nperps.fillna(int(df.nperps.mean()))
df.nperpcap = df.nperpcap.fillna(int(df.nperpcap.mean()))
df.nkill = df.nkill.fillna(int(df.nkill.mean()))
df.nkillus = df.nkillus.fillna(int(df.nkillus.mean()))
df.nkillter = df.nkillter.fillna(int(df.nkillter.mean()))
df.nwound = df.nwound.fillna(int(df.nwound.mean()))
df.nwoundus = df.nwoundus.fillna(int(df.nwoundus.mean()))
df.nwoundte = df.nwoundte.fillna(int(df.nwoundte.mean()))

In [6]:
df.shape
df.head()

(114183, 56)

,eventid,iyear,imonth,iday,extended,country,region,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,multiple,success,suicide,attacktype1,targtype1,targsubtype1,corp1,target1,natlty1,gname,motive,guncertain1,individual,nperps,nperpcap,claimed,weaptype1,weapsubtype1,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propcomment,ishostkid,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY
0,199801010001,1998,1,1,0,34,11,Bujumbura Mairie,Bujumbura,-3.375828,29.364107,1.0,0,The incident occurred at Bujumbura Airport.,01/01/1998: Hutu Rebels attacked a Burundi mil...,1,1,1,1.0,0.0,1,0,2,4,34.0,Burundi Military,Burundi Military at Bujumbura Airport,34.0,Hutu extremists,Unknown,0.0,0,100.0,0.0,0,5,5.0,Unknown,104.0,0.0,100.0,6.0,0.0,0.0,-9,4.0,Unknown,0.0,"“Burundi Rebels, Ex-Rwandan Army Soldiers Blam...",“Burundi--Attack Reported on Bujumbura Airport...,Unknown,CETIS,0,1,0,1
1,199801010002,1998,1,1,0,167,9,Moscow (Federal City),Moscow,55.751377,37.579914,1.0,0,Unknown,"01/01/1998: In Russia, a small bomb hidden in ...",1,1,1,0.0,0.0,1,0,3,19,102.0,Moscow Metro System,An underground passage in the Moscow Metro,167.0,Unknown,Unknown,0.0,0,-76.0,0.0,0,6,14.0,Unknown,0.0,0.0,0.0,3.0,0.0,0.0,0,4.0,Unknown,0.0,"“Bomb injures 3 in Moscow subway system,” The ...","“Bomb injures 3 in Moscow subway,” Charleston ...","“Bomb Injures 3 Workers in Moscow Metro,” Los ...",CETIS,-9,-9,0,-9
2,199801010003,1998,1,1,0,603,8,Northern Ireland,Belfast,54.607712,-5.956210,1.0,0,Belfast (Capital City),01/01/1998: The breakaway Loyalist Volunteer F...,1,1,1,0.0,0.0,1,0,2,14,68.0,Civilian,Eddie Trainor,233.0,Loyalist Volunteer Forces (LVF),Unknown,0.0,0,-76.0,0.0,1,5,2.0,Unknown,1.0,0.0,0.0,0.0,0.0,0.0,0,4.0,Unknown,0.0,“Protestant gunmen kill Catholic in New Year's...,“Ulster Peace Shattered by Shooting: Catholic ...,Unknown,CETIS,0,0,1,1
3,199801020001,1998,1,2,0,95,10,Baghdad,Baghdad,33.303566,44.371773,1.0,0,Unknown,01/02/1998: The United Nations Special Commiss...,1,1,1,0.0,0.0,1,0,3,7,47.0,United Nations,UNSCOM Office Building in Baghdad,999.0,Unknown,It was believed this attack was meant to intim...,0.0,0,-76.0,0.0,0,6,11.0,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,The attack caused damage to a security checkpo...,0.0,“Iraq Condemns Attack on UNSCOM Baghdad Office...,"Farouk Choukri , “Iraq, UN Officials Continue ...","“Iraqi Interior Minister on UNSCOM Attack, Kuw...",CETIS,-9,-9,1,1
4,199801020002,1998,1,2,0,155,10,West Bank,Unknown,31.995965,35.271110,4.0,0,Unknown,01/02/1998: An Israeli woman was critically wo...,1,1,1,0.0,0.0,0,0,2,14,71.0,Civilian,Israeli Civilian,97.0,Unknown,Unknown,0.0,0,-76.0,0.0,0,5,2.0,Unknown,0.0,0.0,0.0,1.0,0.0,0.0,0,4.0,Unknown,0.0,"“Woman Shot,” The Philadelphia Inquirer, Janua...",“Israeli Woman Critically Hurt by Gunfire in W...,Unknown,CETIS,-9,-9,0,-9


In [7]:
# 列分类
zeroone_col = ['extended', 'crit1', 'crit2', 'crit3', 'success', 'suicide', 'weapdetail', 'individual']
onehot_col = ['specificity', 'country', 'region', 'vicinity', 'doubtterr', 'multiple',
              'attacktype1', 'targtype1', 'targsubtype1', 'guncertain1', 'weaptype1', 'weapsubtype1', 'property', 'propextent', 
              'ishostkid', 'dbsource', 'natlty1', 'INT_LOG', 'INT_MISC', 'INT_ANY', 'INT_IDEO', 'claimed']
many_onehot_col = ['corp1', 'target1', 'gname', 'provstate', 'city']
txt_col = ['location', 'summary', 'scite1', 'scite2', 'scite3', 'motive', 'propcomment']
cont_col = ['iyear', 'imonth', 'iday', 'latitude', 'longitude', 'nperps', 'nperpcap', 'nkill', 'nkillus', 'nkillter', 'nwound', 
            'nwoundus', 'nwoundte']

full_col = zeroone_col+onehot_col+many_onehot_col+txt_col+cont_col
len(full_col)

55

In [9]:
df = df[zeroone_col+onehot_col+many_onehot_col+cont_col]
df.shape
df.to_csv('./feature/df_fillna.csv', index=None)

(114183, 48)

## one-hot

In [8]:
for i in tqdm_notebook(onehot_col):
    tmp = pd.get_dummies(df[i], prefix=i)
    df = pd.concat([df, tmp], axis=1)
    df = df.drop([i], axis=1)

A Jupyter Widget

In [10]:
df.shape
df.head()

(114183, 648)

,eventid,iyear,imonth,iday,extended,provstate,city,latitude,longitude,location,summary,crit1,crit2,crit3,success,suicide,corp1,target1,gname,motive,individual,nperps,nperpcap,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,propcomment,scite1,scite2,scite3,specificity_-1.0,specificity_1.0,specificity_2.0,specificity_3.0,specificity_4.0,specificity_5.0,country_4,country_5,country_6,country_8,country_11,country_12,country_14,country_15,country_16,country_17,country_18,country_19,country_21,country_22,country_23,country_25,country_26,country_28,country_30,country_32,country_33,country_34,country_35,country_36,country_37,country_38,country_41,country_42,country_43,country_44,country_45,country_47,country_49,country_50,country_51,country_53,country_54,country_55,country_56,country_58,country_59,country_60,country_62,country_63,country_64,country_65,country_67,country_68,country_69,country_72,country_73,country_74,country_75,country_76,country_78,country_83,country_84,country_85,country_86,country_87,country_88,country_89,country_90,country_91,country_92,country_93,country_94,country_95,country_96,country_97,country_98,country_99,country_100,country_101,country_102,country_103,country_104,country_106,country_107,country_108,country_109,country_110,country_111,country_112,country_113,country_117,country_118,country_119,country_120,country_121,country_122,country_123,country_124,country_128,country_130,country_132,country_136,country_137,country_138,country_139,country_141,country_142,country_144,country_145,country_146,country_147,country_151,country_153,country_155,country_156,country_157,country_158,country_159,country_160,country_161,country_162,country_164,country_166,country_167,country_168,country_173,country_174,country_175,country_177,country_179,country_180,country_181,country_182,country_183,country_184,country_185,country_186,country_190,country_195,country_197,country_198,country_199,country_200,country_201,country_202,country_203,country_204,country_205,country_207,country_208,country_209,country_210,country_213,country_214,country_215,country_217,country_218,country_219,country_222,country_223,country_228,country_229,country_230,country_231,country_235,country_347,country_349,country_422,country_603,country_1001,country_1002,country_1003,country_1004,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9,region_10,region_11,region_12,vicinity_-9,vicinity_0,vicinity_1,doubtterr_-1.0,doubtterr_0.0,doubtterr_1.0,multiple_-1.0,multiple_0.0,multiple_1.0,attacktype1_1,attacktype1_2,attacktype1_3,attacktype1_4,attacktype1_5,attacktype1_6,attacktype1_7,attacktype1_8,attacktype1_9,targtype1_1,targtype1_2,targtype1_3,targtype1_4,targtype1_5,targtype1_6,targtype1_7,targtype1_8,targtype1_9,targtype1_10,targtype1_11,targtype1_12,targtype1_13,targtype1_14,targtype1_15,targtype1_16,targtype1_17,targtype1_18,targtype1_19,targtype1_20,targtype1_21,targtype1_22,targsubtype1_-1.0,targsubtype1_1.0,targsubtype1_2.0,targsubtype1_3.0,targsubtype1_4.0,targsubtype1_5.0,targsubtype1_6.0,targsubtype1_7.0,targsubtype1_8.0,targsubtype1_9.0,targsubtype1_10.0,targsubtype1_11.0,targsubtype1_12.0,targsubtype1_13.0,targsubtype1_14.0,targsubtype1_15.0,targsubtype1_16.0,targsubtype1_17.0,targsubtype1_18.0,targsubtype1_19.0,targsubtype1_20.0,targsubtype1_21.0,targsubtype1_22.0,targsubtype1_23.0,targsubtype1_24.0,targsubtype1_25.0,targsubtype1_26.0,targsubtype1_27.0,targsubtype1_28.0,targsubtype1_29.0,targsubtype1_30.0,targsubtype1_31.0,targsubtype1_32.0,targsubtype1_33.0,targsubtype1_34.0,targsubtype1_35.0,targsubtype1_36.0,targsubtype1_37.0,targsubtype1_39.0,targsubtype1_40.0,targsubtype1_41.0,targsubtype1_42.0,targsubtype1_43.0,targsubtype1_44.0,targsubtype1_45.0,targsubtype1_46.0,targsubtype1_47.0,targsubtype1_48.0,targsubtype1_49.0,targsubtype1_50.0,targsubtype1_51.0,targsubtype1_52.0,targsubtype1_53.0,targsubtype1_54.0,targsubtype1_55.0,targsubtype1_56.0,targsubtype1_57.0,targsubtype1_58.0,targsubtype1_59.0,targsubt

## word2vec

In [9]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

In [10]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [11]:
col = []

for i in tqdm_notebook(txt_col):
    words = []
    for s in df[i]:
        word = [0] * 300
        w_num = 0
        for w in s.split(' '):
            if w.lower() not in stop and w in model:
                word += model[w]
                w_num += 1
            elif w.lower() not in stop and w.lower() in model:
                word += model[w.lower()]
                w_num += 1
        words.append(list(np.array(word) / w_num))
    col.append(words)

A Jupyter Widget

In [12]:
for i, n in tqdm_notebook(enumerate(txt_col)):
    df[n + 'w2v'] = col[i]

A Jupyter Widget

In [13]:
def w2v_pop(x):
    return x.pop()

for n in tqdm_notebook(txt_col):
    for i in range(299):
        df[n + 'w2v_' + str(i)] = df[n + 'w2v'].apply(w2v_pop)

A Jupyter Widget

In [14]:
w2v_col = [i+'w2v' for i in txt_col]
df = df.drop(txt_col+w2v_col, axis=1)

In [15]:
df.to_csv('./feature/df_preprocess.csv', index=None)